# Realtime Face Recogntion using OpenCV

In [1]:
from PIL import Image
from keras.applications.vgg16 import preprocess_input
import base64
from io import BytesIO
import json
import random
import cv2
from keras.models import load_model
import numpy as np
from keras.preprocessing import image

## Load models to be used

In [2]:
# Looadind the previously trained model
model = load_model('facefeatures_new_model.h5')

# Loading the cascades
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

## Draw Rectangle Each face

In [9]:
def draw_boundary(img,classifier,scaleFactor,minNeighbors,color,model):
    #gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(img, scaleFactor, minNeighbors)
    
    coords=[]
    
    for (x,y,w,h) in features:
        cv2.rectangle(img,(x,y),(x+w,y+h),color,2)
        face = img[y:y+h, x:x+w]
        face = cv2.resize(face, (224, 224))
        im = Image.fromarray(face, 'RGB')
           #Resizing into 128x128 because we trained the model with this image size.
        img_array = np.array(im)
                    #Our keras model used a 4D tensor, (images x height x width x channel)
                    #So changing dimension 128x128x3 into 1x128x128x3 
        img_array = np.expand_dims(img_array, axis=0)
        pred = model.predict(img_array)
        
        if (pred[0][0]>0.7):
            name='Kuku'
            cv2.putText(img,name, (x, y-5), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0,255,0), 2)
        elif (pred[0][1]>0.7):
            name='Julienne'
            cv2.putText(img,name, (x, y-5), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0,255,0), 2)
        else:
            cv2.putText(img,'Unknown', (x, y-5), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0,255,0), 2)
        
        coords = [x,y,w,h]
    
    return coords

## Recognize Function

In [10]:
def recognize(img,model,faceCascade):
    coords = draw_boundary(img,faceCascade,1.3,5,(255,255,255),model)
    return img

## Classify Face per Frame

In [11]:
video_capture = cv2.VideoCapture(0)
while True:
    _, frame = video_capture.read()
    img = recognize(frame,model,face_cascade)
    cv2.imshow('Face Recognition', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
        
video_capture.release()
cv2.destroyAllWindows()
